In [1]:
sc.install_pypi_package("pandas==2.2.3")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,application_1747590181230_0007,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Not uninstalling python-dateutil at /usr/lib/python3.9/site-packages, outside environment /mnt/yarn/usercache/livy/appcache/application_1747590181230_0007/container_1747590181230_0007_01_000001/tmp/spark-c10c4305-52e8-4e71-b791-68ad1f6b9c9b
    Can't uninstall 'python-dateutil'. No files were found to uninstall.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.23.11 requires cryptography<40.0.2,>=3.3.2, but you have cryptography 3.2.1 which is incompatible.
awscli 2.23.11 requires python-dateutil<=2.9.0,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.

In [2]:
sc.install_pypi_package("tensorflow==2.15.0")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Attempting uninstall: google-auth
    Found existing installation: google-auth 1.23.0
    Not uninstalling google-auth at /usr/local/lib64/python3.9/site-packages, outside environment /mnt/yarn/usercache/livy/appcache/application_1747590181230_0007/container_1747590181230_0007_01_000001/tmp/spark-c10c4305-52e8-4e71-b791-68ad1f6b9c9b
    Can't uninstall 'google-auth'. No files were found to uninstall.
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.2
    Not uninstalling google-auth-oauthlib at /usr/local/lib64/python3.9/site-packages, outside environment /mnt/yarn/usercache/livy/appcache/application_1747590181230_0007/container_1747590181230_0007_01_000001/tmp/spark-c10c4305-52e8-4e71-b791-68ad1f6b9c9b
    Can't uninstall 'google-auth-oauthlib'. No files were found to uninstall.
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.1
    Not uninstalling tensorboard at /usr/local/lib64/python3.9/si

In [3]:
sc.install_pypi_package("scikit-learn")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sc.install_pypi_package("keras==2.15.0")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [5]:
df_spark = spark.read.parquet("s3://aws-emr-studio-201107706822-us-east-1/1746053900403/e-6HYCE3WANA8IROUS7NBVDSOU4/processed/training_data/training_data_6/")
df_spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[JobID: string, UserID: string, label: int, WindowID: int, user_vec_array: array<double>, job_vec_array: array<double>, user_0: double, job_0: double, user_1: double, job_1: double, user_2: double, job_2: double, user_3: double, job_3: double, user_4: double, job_4: double, user_5: double, job_5: double, user_6: double, job_6: double, user_7: double, job_7: double, user_8: double, job_8: double, user_9: double, job_9: double, user_10: double, job_10: double, user_11: double, job_11: double, user_12: double, job_12: double, user_13: double, job_13: double, user_14: double, job_14: double, user_15: double, job_15: double, user_16: double, job_16: double, user_17: double, job_17: double, user_18: double, job_18: double, user_19: double, job_19: double, user_20: double, job_20: double, user_21: double, job_21: double, user_22: double, job_22: double, user_23: double, job_23: double, user_24: double, job_24: double, user_25: double, job_25: double, user_26: double, job_26: double, 

In [6]:
df = df_spark.limit(1000).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np

# Combine the user and job columns row by row
X = df[user_cols + job_cols].apply(
    lambda row: np.concatenate([np.array(x) if isinstance(x, list) else np.array([x]) for x in row]), axis=1
)

# Now convert to NumPy array (each row is a flat vector)
X = np.stack(X.values)

# Labels (this is likely already fine)
y = df["label"].values

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# Train/val split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
input_dim = X_train.shape[1]

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_dim,)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name="auc")]
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
X_train

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([[ 0.03113851, -0.00964772, -0.00950973, ...,  0.10390097,
         0.03866764,  0.1502898 ],
       [ 0.01453882, -0.03008598,  0.03395284, ...,  0.08840981,
         0.08797282,  0.09977063],
       [ 0.02504479, -0.0093492 ,  0.0120922 , ...,  0.02531434,
        -0.04238244,  0.13309828],
       ...,
       [-0.02279923, -0.01912982,  0.03019753, ..., -0.11175292,
         0.10144247,  0.45917628],
       [ 0.01561314, -0.04328267,  0.00792752, ...,  0.08254002,
         0.03158685,  0.13963719],
       [ 0.02979145, -0.04652837,  0.00527034, ...,  0.01019265,
        -0.0109596 ,  0.15495279]])

In [18]:
X_val

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([[ 0.0262509 , -0.00067337, -0.0196966 , ..., -0.02847424,
         0.03132371,  0.11821058],
       [ 0.02504479, -0.0093492 ,  0.0120922 , ..., -0.13012983,
        -0.11990716,  0.21138105],
       [ 0.02504479, -0.0093492 ,  0.0120922 , ..., -0.11175292,
         0.10144247,  0.45917628],
       ...,
       [-0.00047918, -0.01163639,  0.02591962, ...,  0.00978794,
         0.0589384 ,  0.07353352],
       [ 0.01360871, -0.03026879,  0.00205526, ...,  0.04154324,
         0.07223233,  0.11907196],
       [ 0.04267806,  0.01914843,  0.03182328, ...,  0.07628706,
         0.0238895 ,  0.09330546]])

In [19]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=128,
    epochs=10
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Epoch 1/10
7/7 [==============================] - 1s 41ms/step - loss: 0.6565 - accuracy: 0.6700 - auc: 0.5354 - val_loss: 0.5968 - val_accuracy: 0.7050 - val_auc: 0.6977
Epoch 2/10
7/7 [==============================] - 0s 7ms/step - loss: 0.6026 - accuracy: 0.6888 - auc: 0.6429 - val_loss: 0.5655 - val_accuracy: 0.7050 - val_auc: 0.8470
Epoch 3/10
7/7 [==============================] - 0s 7ms/step - loss: 0.5709 - accuracy: 0.6900 - auc: 0.7460 - val_loss: 0.5370 - val_accuracy: 0.7100 - val_auc: 0.8494
Epoch 4/10
7/7 [==============================] - 0s 7ms/step - loss: 0.5228 - accuracy: 0.7237 - auc: 0.8421 - val_loss: 0.4794 - val_accuracy: 0.7400 - val_auc: 0.8757
Epoch 5/10
7/7 [==============================] - 0s 7ms/step - loss: 0.4632 - accuracy: 0.7575 - auc: 0.8868 - val_loss: 0.4174 - val_accuracy: 0.8300 - val_auc: 0.9050
Epoch 6/10
7/7 [==============================] - 0s 7ms/step - loss: 0.3780 - accuracy: 0.8650 - auc: 0.9323 - val_loss: 0.3212 - val_accuracy: 0.87

In [20]:
loss, accuracy, auc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7/7 [==============================] - 0s 2ms/step - loss: 0.1052 - accuracy: 0.9550 - auc: 0.9972
Validation Accuracy: 0.9550, AUC: 0.9972

In [21]:
model.save("mlp_recommender_tf_model_6")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
sc.install_pypi_package("boto3")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
import boto3
import shutil

# Variables
bucket_name = 'aws-emr-studio-201107706822-us-east-1'
key = '1746053900403/e-6HYCE3WANA8IROUS7NBVDSOU4/processed/models/mlp_recommender_tf_model_6.tar.gz'

# Create archive
shutil.make_archive('mlp_recommender_tf_model_6', 'gztar', 'mlp_recommender_tf_model_6')

# Upload using boto3
s3 = boto3.client('s3')
s3.upload_file('mlp_recommender_tf_model_6.tar.gz', bucket_name, key)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…